In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from func.ml_utils import get_factorize_feature
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
logger = logger_func()

2019-10-02 17:28:27,753 func.utils 347 [INFO]    [logger_func] start 


In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'date']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

# paths_train = glob('../submit/re_sub/50*_train.gz')
# paths_test  = glob('../submit/re_sub/50*_test.gz')
# paths_train += glob('../submit/re_sub/Tran*_train.gz')
# paths_test  += glob('../submit/re_sub/Tran*_test.gz')
# paths_train += glob('../submit/re_sub/is*_train.gz')
# paths_test  += glob('../submit/re_sub/is*_test.gz')

paths_train = glob('../submit/re_sub/*_train.gz')
paths_test  = glob('../submit/re_sub/*_test.gz')
# paths_train += glob('../submit/add_feature/*_train.gz')
# paths_test  += glob('../submit/add_feature/*_test.gz')

print(len(paths_train))
# sys.exit()
# paths_train += glob('../feature/valid_trush/528*uid2*_train.gz')
# paths_test  += glob('../feature/valid_trush/528*uid2*_test.gz')

# for path in paths_train:
#     if path.count('C14_ratio'):
#         paths_train.remove(path)
        
# for path in paths_test:
#     if path.count('C14_ratio'):
#         paths_test.remove(path)
    
# paths_train = glob('../feature/raw_use/*_train.gz')
# paths_test = glob('../feature/raw_use/*_test.gz')
# paths_train = [path for path in paths_train if filter_feature(path) ]
# paths_test = [path for path in paths_test if filter_feature(path) ]

# paths_train_feature = sorted(glob('../feature/org_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/org_use/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/valid/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/valid/*_test.gz'))

# paths_train_feature += sorted(glob('../feature/kernel/*_train.gz'))
# paths_test_feature  += sorted(glob('../feature/kernel/*_test.gz'))

# paths_train_feature = sorted(glob('../feature/valid_use/*_train.gz'))
# paths_test_feature  = sorted(glob('../feature/valid_use/*_test.gz'))
paths_train_feature = []
paths_test_feature  = []

# df_train = reduce_mem_usage( parallel_load_data(paths_train) )
# df_test  = reduce_mem_usage( parallel_load_data(paths_test) )
df_train = parallel_load_data(paths_train)
df_test  = parallel_load_data(paths_test)
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)

283


In [3]:
data = pd.concat([df_train, df_test], axis=0)

cols_categorical = [col for col in df_train.columns 
                    if not col in COLUMNS_IGNORE
                    and 
                    (
                    col.count('uid')
                    or col.count('ugr')
                    or col.count('60')
                    )
                   ]

# list_drop = []
# for col in tqdm(cols_categorical):
#     cnt = df_train[col].head(2000).value_counts().shape[0]
#     if cnt > 100:
#         list_drop.append(col)
#         continue
#     cnt = data[col].value_counts().shape[0]
#     if cnt > 200:
#         list_drop.append(col)
#     else:
#         data[col].fillna(-99999, inplace=True)
            
# cols_categorical = list(set(cols_categorical) -set(list_drop))
# print(len(cols_categorical))

data = get_factorize_feature(data, cols_categorical)
df_train = data.iloc[:len(df_train)]
df_test  = data.iloc[len(df_train):]


# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
# bear = pd.read_csv(same_user_path)
# bear = bear[[COLUMN_ID, 'predicted_user_id']]
# max_id = bear['predicted_user_id'].max()
# bear.loc[bear[bear['predicted_user_id'].isnull()].index, 'predicted_user_id'] = np.arange(
#     bear['predicted_user_id'].isnull().sum() ) + 1 + max_id
# bear['predicted_user_id'] =  bear['predicted_user_id'].astype('int')
# bear.shape

In [4]:
is_submit = [True, False][0]
n_splits = 6
set_type = 'new_set'

valid_paths_train = paths_train_feature[:]
valid_paths_test  = paths_test_feature[:]

#========================================================================
# pathの存在チェック。なぜかたびたびFileNotFoundErrorが起きるので,,,
#========================================================================
remove_paths = []
for trn_path, tes_path in zip(valid_paths_train, valid_paths_test):
    if os.path.exists(trn_path) and os.path.exists(tes_path):
        pass
    else:
        remove_paths.append(trn_path)
        remove_paths.append(tes_path)
for path in remove_paths:
    if path.count('train'):
        valid_paths_train.remove(path)
        print(f'remove {path}')
    elif path.count('test'):
        valid_paths_test.remove(path)
        print(f'remove {path}')

if len(valid_paths_train):
    df_feat_train = parallel_load_data(valid_paths_train)
    df_feat_test  = parallel_load_data(valid_paths_test)
    
    col_drops = eval_check_feature(df_feat_train, df_feat_test)
    
    tmp_train = df_train.join(df_feat_train)
    tmp_test = df_test.join(df_feat_test)
else:
    tmp_train = df_train
    tmp_test = df_test

#========================================================================
# Train Test で片方に存在しないFeatureを除外
#========================================================================
diff_cols = list(set(tmp_train.columns) - set(tmp_test.columns))

for col in list(set(diff_cols)):
    from_dir = 'valid'
    to_dir = 'valid_trush'
    move_feature([col], from_dir, to_dir)
tmp_train.drop(diff_cols, axis=1, inplace=True)
print(f"  * Diff Features: {len(diff_cols)}")

# same_user_path = '../output/same_user_pattern/0902__same_user_id__card_addr_pemail_M.csv'
# tmp_train = tmp_train.merge(bear[[COLUMN_ID, 'predicted_user_id']], how='inner', on=COLUMN_ID)
# COLUMN_GROUP = 'predicted_user_id'
# COLUMNS_IGNORE.append('predicted_user_id')

### DT-M
group_kfold_path = '../input/0908_ieee__DT-M_GroupKFold.gz'
group = read_pkl_gzip(group_kfold_path)
tmp_train[COLUMN_GROUP] = group

# tmp_train[COLUMN_GROUP] = tmp_train['528__ugr_uid3_Regist_date_agg_V95_137_mean_mean'].fillna(0)

#========================================================================
# Features elimination 
#==============================================================
# from scipy.stats import ks_2samp
# features_check = []
# columns_to_check = set(list(tmp_train)).difference(COLUMNS_IGNORE)
# for i in columns_to_check:
#     features_check.append(ks_2samp(tmp_test[i], tmp_train[i])[1])

# features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
# features_discard = list(features_check[features_check==0].index)
# print(features_discard)
# tmp_train.drop(features_discard, axis=1, inplace=True)
# tmp_test.drop(features_discard, axis=1, inplace=True)

  * Diff Features: 0


In [5]:
n_splits = 2
model_type = "cat"
params = {
#     'n_jobs': 96,
    'seed': 1208,
#     'subsample': 0.9,
#     'boosting_type':'Ordered',
#     'num_boost_round':3000,
#     'random_seed': 1208,
#     'has_time': True,
#     'max_depth': 10,
    'loss_function': 'Logloss',
    'custom_loss':['AUC'],
#     'logging_level':'Silent',
    'task_type' : 'CPU',
    'early_stopping_rounds' : 100,
    'n_splits': n_splits,
    'model_type': model_type,
    'fold': ['stratified', 'group'][1],
    'verbose': 10,
}

if is_submit:
    pass
#     params['learning_rate'] = 0.01
#     params['learning_rate'] = 0.02
    params['learning_rate'] = 0.1
#     params["early_stopping_rounds"] = 100
#     params["early_stopping_rounds"] = 70
#     params["early_stopping_rounds"] = 10

use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE]
logger.info(f"* EXP: dataset {set_type} {tmp_train.shape} lr {params['learning_rate']} ")
            
if False:
#     sample = tmp_train.sample(10000)
#     sample[COLUMN_GROUP].value_counts()
    use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE][:10]
    params['early_stopping_rounds'] = 1
    params['num_boost_round'] = 1

feim, _ = eval_train(
    logger,
    tmp_train.head(200000),
    Y.head(200000),
    tmp_test,
#     sample,
#     Y.head(10000),
#     tmp_test.head(1000),
    COLUMN_GROUP,
    model_type,
    params,
    is_adv=[True, False][1],
    is_viz=[True, False][1],
    cols_categorical=cols_categorical
)
feim = list_result_feim[0]

2019-10-02 17:29:20,257 func.utils 33 [INFO]    [<module>] * EXP: dataset new_set (590540, 284) lr 0.1  


0:	learn: 0.5358544	test: 0.5463536	best: 0.5463536 (0)	total: 4.55s	remaining: 1h 15m 47s
10:	learn: 0.1192049	test: 0.1305528	best: 0.1305528 (10)	total: 35.8s	remaining: 53m 36s
20:	learn: 0.0787675	test: 0.0965824	best: 0.0965824 (20)	total: 2m 7s	remaining: 1h 38m 58s
30:	learn: 0.0678082	test: 0.0913108	best: 0.0913071 (29)	total: 4m 20s	remaining: 2h 15m 53s
40:	learn: 0.0642730	test: 0.0887717	best: 0.0887717 (40)	total: 5m 50s	remaining: 2h 16m 49s
50:	learn: 0.0616090	test: 0.0881611	best: 0.0881611 (50)	total: 7m 32s	remaining: 2h 20m 25s
60:	learn: 0.0601159	test: 0.0883102	best: 0.0877956 (51)	total: 9m	remaining: 2h 18m 41s
70:	learn: 0.0587840	test: 0.0895615	best: 0.0877956 (51)	total: 10m 19s	remaining: 2h 15m 7s
80:	learn: 0.0579975	test: 0.0901292	best: 0.0877956 (51)	total: 11m 26s	remaining: 2h 9m 46s
90:	learn: 0.0572771	test: 0.0902935	best: 0.0877956 (51)	total: 12m 32s	remaining: 2h 5m 19s
100:	learn: 0.0562301	test: 0.0908868	best: 0.0877956 (51)	total: 13m 42

2019-10-02 17:48:58,636 func.utils 168 [INFO]    [ieee_cv]   * Fold0 2017-12: 0.7066599736839209 | Bear's...PB:0.5757470312921227 PV:0.5577695602903563 All:0.5635048862294244 


[  * Fold0 Validation-DT-M 2017-12: 134339] done in 1177 s
0:	learn: 0.5253337	test: 0.5310087	best: 0.5310087 (0)	total: 3.14s	remaining: 52m 20s
10:	learn: 0.0996639	test: 0.1307262	best: 0.1307262 (10)	total: 27.6s	remaining: 41m 20s
20:	learn: 0.0651289	test: 0.1090729	best: 0.1090729 (20)	total: 1m 53s	remaining: 1h 28m 17s
30:	learn: 0.0556728	test: 0.1062806	best: 0.1062806 (30)	total: 4m 21s	remaining: 2h 16m 10s
40:	learn: 0.0519541	test: 0.1037534	best: 0.1037534 (40)	total: 6m 53s	remaining: 2h 41m 12s
50:	learn: 0.0503892	test: 0.1019317	best: 0.1019317 (50)	total: 8m 21s	remaining: 2h 35m 40s
60:	learn: 0.0491274	test: 0.1004453	best: 0.1004453 (60)	total: 10m 4s	remaining: 2h 35m 7s
70:	learn: 0.0481661	test: 0.0991379	best: 0.0991379 (70)	total: 11m 44s	remaining: 2h 33m 34s
80:	learn: 0.0472131	test: 0.0977395	best: 0.0977395 (80)	total: 13m 11s	remaining: 2h 29m 39s
90:	learn: 0.0465834	test: 0.0970463	best: 0.0970463 (90)	total: 14m 22s	remaining: 2h 23m 40s
100:	lear

2019-10-02 19:49:02,906 func.utils 168 [INFO]    [ieee_cv]   * Fold1 2018-1: 0.7015672045849004 | Bear's...PB:0.573427644759766 PV:0.5805814934620687 All:0.5782564634979782 


[  * Make Prediction Result File.] done in 32 s
* Check Corr with Past Submit.
  *  (506691,) (506691,)
  * LB0.9566 / 0.8392174694787562
  *  (506691,) (506691,)
  * LB0.9609 / 0.8400032836260126


2019-10-02 19:49:36,810 func.utils 361 [INFO]    [eval_train] * CV: 0.7041135891344106 | BestIter: 1.0 


  *  (506691,) (506691,)
  * LB0.9566 / -0.0004251996722853398


IndexError: list index out of range